In [1]:
from rdflib import Graph
from rdflib.extras.external_graph_libs import rdflib_to_networkx_graph
import networkx as nx
from networkx.algorithms.shortest_paths.generic import all_shortest_paths
from rdflib import URIRef

In [49]:
def get_neighbours(G, edge, n=3):
    """return the first n neighbours of a given node
    """
    return [i[:] for i in nx.neighbors(G, edge[0]) if (i[:] != edge[1][:]) and (i[:].startswith('http'))][:n]

In [9]:
path = "./hc_dump_latest-wdt&foaf-20211117-151212.nt"
rg = Graph().parse(path, format='nt')
print("rdflib Graph loaded successfully with {} triples".format(len(rg)))

rdflib Graph loaded successfully with 2199906 triples


In [10]:
G = rdflib_to_networkx_graph(rg)
print("networkx Graph loaded successfully with length {}".format(len(G)))

networkx Graph loaded successfully with length 1187202


In [16]:
ent_a = URIRef("http://collections.vam.ac.uk/item/O1389838")
ent_b = URIRef("https://collection.sciencemuseumgroup.org.uk/objects/co8602393")

In [14]:
all_sps = all_shortest_paths(G, ent_a, ent_b)
path_graphs = [nx.path_graph(sp) for sp in all_sps]

In [22]:
# choose a path
path = path_graphs[0]


In [64]:
# list of dicts, where key = each step of the best path
# within each dict is next-best-step, and 3 (random) connected 'neighbour' nodes
path_edges = [i for i in path.edges()]

path_json = []
for edge in path_edges:
    path_json.append({
        edge[0][:]: {
        'next-best-step': edge[1][:],
        'neighbours': get_neighbours(G, edge),
        }}
    )
path_json.append({
    path_edges[-1][1][:]: {
        'next-best-step': 'END',
        'neighbours': [],
    }
})
path_json

[{'http://collections.vam.ac.uk/item/O1389838': {'next-best-step': 'https://api.vam.ac.uk/v2/objects/search?id_technique=AAT53271',
   'neighbours': ['https://api.vam.ac.uk/v2/objects/search?id_technique=AAT131119',
    'https://api.vam.ac.uk/v2/objects/search?id_person=A27048',
    'https://api.vam.ac.uk/v2/objects/search?id_technique=AAT53319']}},
 {'https://api.vam.ac.uk/v2/objects/search?id_technique=AAT53271': {'next-best-step': 'http://collections.vam.ac.uk/item/O1519245',
   'neighbours': ['http://collections.vam.ac.uk/item/O1286466',
    'http://collections.vam.ac.uk/item/O691668',
    'http://collections.vam.ac.uk/item/O854598']}},
 {'http://collections.vam.ac.uk/item/O1519245': {'next-best-step': 'made in england',
   'neighbours': ['https://api.vam.ac.uk/v2/objects/search?id_organisation=AUTH355220',
    'https://api.vam.ac.uk/v2/objects/search?id_organisation=AUTH355221',
    'https://api.vam.ac.uk/v2/objects/search?id_technique=AAT53271']}},
 {'made in england': {'next-bes

In [58]:
path_edges[-1][1][:]

'https://collection.sciencemuseumgroup.org.uk/objects/co8602393'

In [52]:
path_json.keys()

dict_keys(['http://collections.vam.ac.uk/item/O1389838', 'https://api.vam.ac.uk/v2/objects/search?id_technique=AAT53271', 'http://collections.vam.ac.uk/item/O1519245', 'made in england', 'https://collection.sciencemuseumgroup.org.uk/objects/co8010891', 'https://collection.sciencemuseumgroup.org.uk/people/cp21665'])